In [6]:
%matplotlib inline


In [7]:
import tensorflow as tf
import keras
from keras import backend as K
from PIL import Image
from keras.preprocessing import image
print(keras.__version__)

2.4.3


In [ ]:
from keras.applications.vgg16 import VGG16

vgg16 = VGG16(include_top=False, weights = 'imagenet', input_shape=(224, 224, 3))
vgg16.summary()

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.optimizers import SGD, RMSprop, Adam

def build_transfer_model(vgg16):
    model =  Sequential(vgg16.layers)
     
    for layer in model.layers[:15]:
           layer.trainable = True
            
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7,activation = 'softmax')) 
        
    return model
        
model = build_transfer_model(vgg16)

In [10]:

model.compile(
    loss = 'categorical_crossentropy',
    optimizer=Adam(lr=0.01),  
    metrics = ['accuracy']
)

In [ ]:
model.summary()

In [13]:
from keras.preprocessing.image import ImageDataGenerator

idg_train = ImageDataGenerator(
    rescale=1/255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=60,
    shear_range=0.1,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
)

idg_validation = ImageDataGenerator(
    rescale=1/255
)

In [ ]:
classes=['0a3','0b2','0c1','a0','a1','a2','a3']


img_itr_train= idg_train.flow_from_directory(
    'E:\train',
    #training data folder
    target_size=(224, 224),
    batch_size=32,
    classes=classes,
    class_mode='categorical',
    shuffle=True
)


img_itr_validation = idg_validation.flow_from_directory(
    'E:\validation',
    #validation data folder
    target_size=(224, 224),
    batch_size=32,
    classes=classes,
    class_mode='categorical'
)

In [ ]:
import os
from datetime import datetime
model_dir = os.path.join(
    'models',
    datetime.now().strftime('%y%m%d_%H%M')
)

os.makedirs(model_dir, exist_ok=True)
            
dir_weights = os.path.join(model_dir, 'weights')
os.makedirs(dir_weights, exist_ok=True)
            

In [10]:
import json
import pickle

model_json = os.path.join(model_dir, 'model.json')
with open(model_json, 'w') as f:
    json.dump(model.to_json(),f)
        
model_classes = os.path.join(model_dir, 'classes.pkl')
with open(model_classes, 'wb') as f:
    pickle.dump(img_itr_train.class_indices, f)

In [ ]:
import math

batch_size = 32
steps_per_epoch = math.ceil(
    img_itr_train.samples/batch_size
)

validation_steps = math.ceil(
    img_itr_validation.samples/batch_size
)


In [12]:
from keras.callbacks import ModelCheckpoint, CSVLogger

cp_filepath = os.path.join(dir_weights, 'ep_{epoch:02d}_ls_{loss:.1f}.h5')
cp = ModelCheckpoint(
                    cp_filepath,
                    monitor='val_acc',
                    verbose=0,
                    save_best_only=True,
                    save_weights_only=False,
                    mode='max',
                    period =10
                    )

csv_filepath = os.path.join(model_dir, 'loss.csv')
csv = CSVLogger(csv_filepath, append=True)

In [ ]:
n_epoch = 100

history = model.fit_generator(
    img_itr_train,
    steps_per_epoch=steps_per_epoch,
    epochs=n_epoch,
    validation_data=img_itr_validation,
    validation_steps=validation_steps,
    callbacks = [cp, csv]
    
)

model.save(os.path.join(model_dir,'TI_model.h5'))

In [ ]:
#predict by confusion matrix

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
sess = tf.Session(config=config)
K.set_session(sess)
import keras
import os
import glob
import numpy as np
from PIL import Image
from keras.utils import np_utils
from keras.models import Sequential, load_model
from keras.preprocessing.image import array_to_img, img_to_array, list_pictures, load_img
from sklearn.metrics import confusion_matrix

path='E:\ '
# test data path
folders=glob.glob(path+'/**', recursive=False)
category_number=len(folders)
categoryname=[]

model=load_model('TI_model.h5')

X =[]
Y = []
Fnames=[]
size=224 #image size

for category_num, folder in enumerate(folders):
    categoryname.append(os.path.basename(folder))
    print(category_num)
    
    files=glob.glob(folder+"/*png")
    for filepath in files:
        
        img=Image.open(filepath)
        img=img.convert("RGB")
        img=img.resize((size,size))
        img=np.asarray(img)
        X.append(img)
        Y.append(category_num) 
        Fnames.append(os.path.basename(filepath))
    
   
X = np.asarray(X)
Y = np.asarray(Y)

X = X.astype('float32')/255
Y = Y.astype('int32')

Y = np_utils.to_categorical(Y, category_number) 

#predict
print('predicting')
probs=model.predict(X, verbose=1)
label=probs.argmax(axis=1)


#confusion matrix
predict_classes = model.predict_classes(X, batch_size=32)
true_classes = np.argmax(Y,1)
print(confusion_matrix(true_classes, predict_classes))

